In [1]:
import numpy as np
import pandas as pd
import gc
import time
import lightgbm as lgb
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import KFold, StratifiedKFold
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import base64
import os
from sklearn.preprocessing import StandardScaler
import datetime

In [2]:
SEED = 42

In [3]:
def shuffle_features(df, target, seed):
    _seed = seed
    _df = pd.DataFrame()
    for column in df.columns:
        _df[column] = df[column].sample(frac=1,random_state=_seed).values
        _seed += 1
    _df["target"] = target
    return _df

In [17]:
print('Load Train Data.')
train_df = pd.read_csv('../input/train.csv')
test_df = pd.read_csv('../input/test.csv')
print('\nShape of Train Data: {}\t Shape of Test Data: {}'
    .format(train_df.shape, test_df.shape))

train_labels = np.array(train_df['target'])
train_index = np.array(train_df.index)

Load Train Data.

Shape of Train Data: (200000, 202)	 Shape of Test Data: (200000, 201)


In [18]:
train_df.drop(['ID_code'], axis=1, inplace=True)
test_df.drop(['ID_code'], axis=1, inplace=True)

In [19]:
train_df.shape

(200000, 201)

In [20]:
train_df_0 = train_df[train_df["target"]==0].drop(columns=["target"])
train_df_1 = train_df[train_df["target"]==1].drop(columns=["target"])

In [21]:
train_df_1.shape, train_df_0.shape

((20098, 200), (179902, 200))

In [22]:
train_df.head()

,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
0,0,8.9255,-6.7863,11.9081,5.0930,11.4607,-9.2834,5.1187,18.6266,-4.9200,...,4.4354,3.9642,3.1364,1.6910,18.5227,-2.3978,7.8784,8.5635,12.7803,-1.0914
1,0,11.5006,-4.1473,13.8588,5.3890,12.3622,7.0433,5.6208,16.5338,3.1468,...,7.6421,7.7214,2.5837,10.9516,15.4305,2.0339,8.1267,8.7889,18.3560,1.9518
2,0,8.6093,-2.7457,12.0805,7.8928,10.5825,-9.0837,6.9427,14.6155,-4.9193,...,2.9057,9.7905,1.6704,1.6858,21.6042,3.1417,-6.5213,8.2675,14.7222,0.3965
3,0,11.0604,-2.1518,8.9522,7.1957,12.5846,-1.8361,5.8428,14.9250,-5.8609,...,4.4666,4.7433,0.7178,1.4214,23.0347,-1.2706,-2.9275,10.2922,17.9697,-8.9996
4,0,9.8369,-1.4834,12.8746,6.6375,12.2772,2.4486,5.9405,19.2514,6.2654,...,-1.4905,9.5214,-0.1508,9.1942,13.2876,-1.5121,3.9267,9.5031,17.9974,-8.8104


In [23]:
train_df["target"].value_counts()

0    179902
1     20098
Name: target, dtype: int64

In [24]:
for i in range(8):
    train_df = pd.concat([train_df, shuffle_features(train_df_1, 1, SEED+10*i)], axis=0, sort=False)

In [25]:
train_df = train_df.sample(frac=1).reset_index(drop=True)

In [28]:
train_df.shape

(360784, 201)

In [27]:
train_df["target"].value_counts()

1    180882
0    179902
Name: target, dtype: int64

In [36]:
__df = []
for i in range(8):
    __df.append(shuffle_features(train_df_1, 1, SEED+10*i))
    print(__df[i].shape)

(20098, 201)
(20098, 201)
(20098, 201)
(20098, 201)
(20098, 201)
(20098, 201)
(20098, 201)
(20098, 201)
